In [217]:
import pandas as pd

In [218]:
cities_df

,city_id,city,country,country_iso,coordination,lat,long
0,0,London,England,GB,51.50722°N 0.12750°W,51.507220,-0.127500
1,1,Paris,France,FR,48.856613°N 2.352222°E,48.856613,2.352222
2,2,Berlin,Germany,DE,52.52000°N 13.40500°E,52.520000,13.405000
3,3,Moscow,Russia,RU,55.75583°N 37.61722°E,55.755830,37.617220
4,4,Istanbul,Turkey,TR,41.01361°N 28.95500°E,41.013610,28.955000
5,5,Barcelona,Spain,ES,41.383°N 2.183°E,41.383000,2.183000
6,6,Saint Petersburg,Russia,RU,59.93750°N 30.30861°E,59.937500,30.308610
7,7,Madrid,Spain,ES,40.41667°N 3.70250°W,40.416670,-3.702500
8,8,Kyiv,Ukraine,UA,50.45000°N 30.52333°E,50.450000,30.523330
9,9,Rome,Italy[a],IT,41.89333°N 12.48278°E,41.893330,12.482780


## Read data from CSV

In [219]:
DISTANCE_AIRPORT_CITYCENTER = 30

In [220]:
cities_df = pd.read_pickle("../01_cities\out_dfs\cities_2.pkl")

In [240]:
df= pd.read_csv("airports.csv")

In [ ]:
df=df.drop(["scheduled_service","gps_code","iata_code","local_code","home_link","wikipedia_link","keywords"],axis=1)

In [246]:
df.type.unique()

array(['heliport', 'small_airport', 'closed', 'seaplane_base',
       'balloonport', 'medium_airport', 'large_airport'], dtype=object)

In [247]:
df=df.loc[~df.type.isin(["heliport","closed","seaplane_base","balloonport"])]

In [224]:
#df = df.loc[df.continent == "EU"]

In [248]:
df = df.loc[df.iso_country.isin(cities_df.country_iso)]

In [204]:
df

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality
12062,334039,AT-0009,small_airport,Ameis Airstrip,48.657837,16.540234,NaN,EU,AT,AT-3,Ameis
14843,34946,BY-0001,medium_airport,Balbasovo Air Base,54.439999,30.296700,620.0,EU,BY,BY-VI,Orsha
14845,34952,BY-0003,medium_airport,Bobrovichi Air Base,52.278301,29.360001,440.0,EU,BY,BY-HO,Kalinovichi
14846,34957,BY-0004,small_airport,Borovtsy Air Base,55.608299,28.678301,456.0,EU,BY,BY-VI,Polatsk
14848,44363,BY-0006,medium_airport,Osovtsy Air Base,52.556999,24.884001,NaN,EU,BY,BY-BR,Byaroza
...,...,...,...,...,...,...,...,...,...,...,...
65718,41917,XPLO,small_airport,Plockton Airstrip,57.335499,-5.673080,226.0,EU,GB,GB-SCT,Plockton
65720,35017,XRWL,medium_airport,Lebyazhye Air Base,50.201698,45.208302,381.0,EU,RU,RU-VGG,Kamyshin
65821,44992,XUBS,medium_airport,Smolensk North Airport,54.824000,32.025000,820.0,EU,RU,RU-SMO,Smolensk
65823,44765,XWPD,small_airport,Sosnovka Airfield,53.261002,45.277000,NaN,EU,RU,RU-PNZ,Penza


In [249]:

# Python 3 program to calculate Distance Between Two Points on Earth
from math import radians, cos, sin, asin, sqrt
def distance_km(row,lat2,lon2,city_id):
     
    # The math module contains a function named
    # radians which converts from degrees to radians.
    #row contains location of airport
    lat1=row.latitude_deg
    lon1=row.longitude_deg
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
 
    c = 2 * asin(sqrt(a))
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
    dist_km = c * r
    if dist_km <= DISTANCE_AIRPORT_CITYCENTER :
        ret = city_id+1
    else:
        ret = 0
    # calculate the result
    return ret

In [251]:
cities_df.shape[0]

13

In [252]:
df["city_id"]= 0
for i in range(cities_df.shape[0]):
    dist_accepted=df.apply(distance_km,axis=1,result_type="expand",args=(cities_df.lat[i],cities_df.long[i],cities_df.city_id[i]))
    #df_dis["dist"+str(i)] = (dist_km <= DISTANCE_AIRPORT_CITYCENTER)
    df["city_id"] += dist_accepted

In [253]:
df.city_id.unique()

array([ 0, 12,  3,  1,  8, 10,  6,  2, 13, 11,  5,  7,  4,  9],
      dtype=int64)

In [254]:
airport_df = df.loc[df.city_id > 0]

In [255]:
airport_df.city_id -=1

C:\Users\Jacqueline\AppData\Local\Temp\ipykernel_22404\2985782767.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airport_df.city_id -=1


In [256]:
airport_df.city_id.max()

12

In [257]:
airport_df=airport_df[["ident","municipality","city_id"]]

In [266]:
airport_df.loc[airport_df.city_id == 7]

,ident,municipality,city_id
21678,ES-0111,Villanueva del Pardillo,7
21715,ES-0148,Villanueva de la Cañada,7
21729,ES-0162,El Molar,7
21745,ES-0178,Brunete,7
21776,ES-0209,Griñón,7
36960,LECU,Madrid,7
36980,LEGT,Getafe,7
37010,LEMD,Madrid,7
37070,LETO,Madrid,7
